## Analisis de Datos

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset=pd.read_csv('../python-ml-course/datasets/wine/winequality-red.csv',sep=';')
dataset.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
dataset.shape

(1599, 12)

### Normalizar dataset

In [4]:
data=(dataset-dataset.min())/(dataset.max()-dataset.min())
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385,0.4
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385,0.4
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385,0.6
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846,0.4


## Obtener numero optimo de clusters

### Metodo  Propagacion de la Afinidad

In [5]:
from sklearn.cluster import AffinityPropagation

In [31]:
af=AffinityPropagation(preference=-50).fit(data)

cluster_center_ids = af.cluster_centers_indices_

n_clust = len(cluster_center_ids)
n_clust

3

### Metodo con Scipy

In [7]:
from scipy.cluster.hierarchy import linkage,dendrogram
from scipy.cluster.hierarchy import fcluster

In [8]:
z=linkage(data,'ward')
z2=z[:,2] #se obtienen todas las distancias de los clusters calculados

def max_diff(dist_clusters):
    diff_clusters=np.diff(dist_clusters)
    max=0
    for index_i,i in enumerate(diff_clusters):
        if(i>max):
            max=i
            idx=index_i
        for index_j,j in enumerate(diff_clusters):
            if(j>max):
                max=j
                idx=index_j          
    return (dist_clusters[idx])

max_d=round(max_diff(z2),2)
max_d
clusters=fcluster(z,max_d,criterion='distance')
print('Numero de clusters optimos: '+ str(clusters.max()))

Numero de clusters optimos: 3


In [24]:
print(len(z))

1598


### Metodo de la silueta

In [9]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples,silhouette_score

In [10]:
max_clusters=len(data)
clusters=range(2,max_clusters) #clusters a partir de 2
resultado={}

In [11]:
for k in clusters:
    kmean_model=KMeans(n_clusters=k).fit(data)
    labels=kmean_model.labels_
    
    siluetas_temp=[]
    
    if 1<k<len(data): #Por seguridad no se crean siluetas con clusters menores a 2 y mayores a len(data)
        
        silhouette_avg = silhouette_score(data, labels)
        print("* Para el numero de clusters = ",k, " el promedio de las siluetas es de :",silhouette_avg)

        sample_silhouette_values = silhouette_samples(data, labels) #valores de cada silueta
          
        for i in range(k):
            ith_cluster_sv = sample_silhouette_values[labels == i]
            print("   - Para el Cluster ", i+1, " su silueta vale : ", np.mean(ith_cluster_sv))
            
            siluetas_temp.append(np.mean(ith_cluster_sv))
            
    resultado[k]=abs((np.max(siluetas_temp)-np.min(siluetas_temp)))

* Para el numero de clusters =  2  el promedio de las siluetas es de : 0.2340788219019556
   - Para el Cluster  1  su silueta vale :  0.281944513451089
   - Para el Cluster  2  su silueta vale :  0.16045682966209815
* Para el numero de clusters =  3  el promedio de las siluetas es de : 0.20490325173788893
   - Para el Cluster  1  su silueta vale :  0.16031581895305436
   - Para el Cluster  2  su silueta vale :  0.1515713311575713
   - Para el Cluster  3  su silueta vale :  0.25388152068735875
* Para el numero de clusters =  4  el promedio de las siluetas es de : 0.18828933079924642
   - Para el Cluster  1  su silueta vale :  0.10372640164013183
   - Para el Cluster  2  su silueta vale :  0.2629879090384539
   - Para el Cluster  3  su silueta vale :  0.19219726583073812
   - Para el Cluster  4  su silueta vale :  0.13710158668694541
* Para el numero de clusters =  5  el promedio de las siluetas es de : 0.179854891786704
   - Para el Cluster  1  su silueta vale :  0.15626046721808312
   

* Para el numero de clusters =  16  el promedio de las siluetas es de : 0.12988812194529958
   - Para el Cluster  1  su silueta vale :  0.1532359776825849
   - Para el Cluster  2  su silueta vale :  0.04524970688504099
   - Para el Cluster  3  su silueta vale :  0.11745199891987666
   - Para el Cluster  4  su silueta vale :  0.18204505696411483
   - Para el Cluster  5  su silueta vale :  0.20693908862231541
   - Para el Cluster  6  su silueta vale :  0.10389649681401872
   - Para el Cluster  7  su silueta vale :  0.24662054148826737
   - Para el Cluster  8  su silueta vale :  0.1955422062426497
   - Para el Cluster  9  su silueta vale :  0.11792091671096089
   - Para el Cluster  10  su silueta vale :  0.08756910841399392
   - Para el Cluster  11  su silueta vale :  0.1438178185347064
   - Para el Cluster  12  su silueta vale :  0.08233837635805215
   - Para el Cluster  13  su silueta vale :  0.15018227475514992
   - Para el Cluster  14  su silueta vale :  0.06411240373852645
   - Para 

* Para el numero de clusters =  23  el promedio de las siluetas es de : 0.13835418974804392
   - Para el Cluster  1  su silueta vale :  0.1428952091087803
   - Para el Cluster  2  su silueta vale :  0.14482291874235548
   - Para el Cluster  3  su silueta vale :  0.11626041197504539
   - Para el Cluster  4  su silueta vale :  0.1350812988528669
   - Para el Cluster  5  su silueta vale :  0.14067464587102835
   - Para el Cluster  6  su silueta vale :  0.24309432603379166
   - Para el Cluster  7  su silueta vale :  0.1479235740165427
   - Para el Cluster  8  su silueta vale :  0.3584913684314474
   - Para el Cluster  9  su silueta vale :  0.08051110014008082
   - Para el Cluster  10  su silueta vale :  0.03546584639244062
   - Para el Cluster  11  su silueta vale :  0.09181751257830882
   - Para el Cluster  12  su silueta vale :  0.0408539677800394
   - Para el Cluster  13  su silueta vale :  0.16619302532446484
   - Para el Cluster  14  su silueta vale :  0.1929609193302231
   - Para el 

* Para el numero de clusters =  28  el promedio de las siluetas es de : 0.13808190956898847
   - Para el Cluster  1  su silueta vale :  0.10209011287562456
   - Para el Cluster  2  su silueta vale :  0.13360791075906103
   - Para el Cluster  3  su silueta vale :  0.06931427151121868
   - Para el Cluster  4  su silueta vale :  0.17393558415674756
   - Para el Cluster  5  su silueta vale :  0.16520541894538193
   - Para el Cluster  6  su silueta vale :  0.17439183413744172
   - Para el Cluster  7  su silueta vale :  0.1514381458099505
   - Para el Cluster  8  su silueta vale :  0.21686193624859434
   - Para el Cluster  9  su silueta vale :  0.12311295746522334
   - Para el Cluster  10  su silueta vale :  0.08036159264750573
   - Para el Cluster  11  su silueta vale :  0.041219874329878864
   - Para el Cluster  12  su silueta vale :  0.11348811852199572
   - Para el Cluster  13  su silueta vale :  0.35110356388825914
   - Para el Cluster  14  su silueta vale :  0.20599521243649085
   - Pa

* Para el numero de clusters =  33  el promedio de las siluetas es de : 0.13450626734513269
   - Para el Cluster  1  su silueta vale :  0.11695509296395538
   - Para el Cluster  2  su silueta vale :  0.1436520358777033
   - Para el Cluster  3  su silueta vale :  0.09581262106234104
   - Para el Cluster  4  su silueta vale :  0.17791074353085756
   - Para el Cluster  5  su silueta vale :  0.11746163546792988
   - Para el Cluster  6  su silueta vale :  0.1473864673440873
   - Para el Cluster  7  su silueta vale :  0.13840531956378704
   - Para el Cluster  8  su silueta vale :  0.17738702004596563
   - Para el Cluster  9  su silueta vale :  0.13734620675200374
   - Para el Cluster  10  su silueta vale :  0.0873585413622519
   - Para el Cluster  11  su silueta vale :  0.111037573797527
   - Para el Cluster  12  su silueta vale :  0.1689938084253672
   - Para el Cluster  13  su silueta vale :  0.1512244991712316
   - Para el Cluster  14  su silueta vale :  0.11534987833338071
   - Para el C

...

* Para el numero de clusters =  139  el promedio de las siluetas es de : 0.16449171483222752
   - Para el Cluster  1  su silueta vale :  0.12944248811380263
   - Para el Cluster  2  su silueta vale :  0.42812531540262794
   - Para el Cluster  3  su silueta vale :  0.1914028067183084
   - Para el Cluster  4  su silueta vale :  0.18968500440582356
   - Para el Cluster  5  su silueta vale :  0.0550835356366265
   - Para el Cluster  6  su silueta vale :  0.17853923301334568
   - Para el Cluster  7  su silueta vale :  0.2186667523498865
   - Para el Cluster  8  su silueta vale :  0.4161227233272004
   - Para el Cluster  9  su silueta vale :  0.18796989925026156
   - Para el Cluster  10  su silueta vale :  0.19782399936667422
   - Para el Cluster  11  su silueta vale :  0.16249555617960598
   - Para el Cluster  12  su silueta vale :  0.41672393345851544
   - Para el Cluster  13  su silueta vale :  0.08873168191855323
   - Para el Cluster  14  su silueta vale :  0.5633426193441384
   - Para e

* Para el numero de clusters =  140  el promedio de las siluetas es de : 0.16651665555715217
   - Para el Cluster  1  su silueta vale :  0.0
   - Para el Cluster  2  su silueta vale :  0.13146333376512978
   - Para el Cluster  3  su silueta vale :  0.042853109292821046
   - Para el Cluster  4  su silueta vale :  0.11335557569962132
   - Para el Cluster  5  su silueta vale :  0.20185410648024235
   - Para el Cluster  6  su silueta vale :  0.07366469586469687
   - Para el Cluster  7  su silueta vale :  0.12315255681827957
   - Para el Cluster  8  su silueta vale :  0.3764424965627292
   - Para el Cluster  9  su silueta vale :  0.24421165571951148
   - Para el Cluster  10  su silueta vale :  0.18937495202472893
   - Para el Cluster  11  su silueta vale :  0.10936104700669384
   - Para el Cluster  12  su silueta vale :  0.15226583157064963
   - Para el Cluster  13  su silueta vale :  0.1874566128747782
   - Para el Cluster  14  su silueta vale :  0.31970333150912356
   - Para el Cluster  1

* Para el numero de clusters =  141  el promedio de las siluetas es de : 0.1606139959918392
   - Para el Cluster  1  su silueta vale :  0.24193069554661892
   - Para el Cluster  2  su silueta vale :  0.23286728550394722
   - Para el Cluster  3  su silueta vale :  0.13017294721718844
   - Para el Cluster  4  su silueta vale :  0.10235055512164319
   - Para el Cluster  5  su silueta vale :  0.11657780823599949
   - Para el Cluster  6  su silueta vale :  0.12697431763856515
   - Para el Cluster  7  su silueta vale :  0.1522966701950971
   - Para el Cluster  8  su silueta vale :  0.24124745405716397
   - Para el Cluster  9  su silueta vale :  0.2808373391699211
   - Para el Cluster  10  su silueta vale :  0.07411890913542062
   - Para el Cluster  11  su silueta vale :  0.0
   - Para el Cluster  12  su silueta vale :  0.14984033192166776
   - Para el Cluster  13  su silueta vale :  0.2810748124760415
   - Para el Cluster  14  su silueta vale :  -0.025020508805138286
   - Para el Cluster  15

* Para el numero de clusters =  142  el promedio de las siluetas es de : 0.1635235407212326
   - Para el Cluster  1  su silueta vale :  0.06956810550920055
   - Para el Cluster  2  su silueta vale :  0.16399303334207696
   - Para el Cluster  3  su silueta vale :  0.07297686505206781
   - Para el Cluster  4  su silueta vale :  -0.030744685308865656
   - Para el Cluster  5  su silueta vale :  0.16451289795613672
   - Para el Cluster  6  su silueta vale :  0.15424139628580927
   - Para el Cluster  7  su silueta vale :  0.22970775926292739
   - Para el Cluster  8  su silueta vale :  0.1762736738839831
   - Para el Cluster  9  su silueta vale :  0.11876683935451872
   - Para el Cluster  10  su silueta vale :  0.10912366472577534
   - Para el Cluster  11  su silueta vale :  0.1934474287590769
   - Para el Cluster  12  su silueta vale :  0.06200960561577424
   - Para el Cluster  13  su silueta vale :  0.17132196279664266
   - Para el Cluster  14  su silueta vale :  0.24713821643052702
   - Pa

KeyboardInterrupt: 

In [12]:
print(len(resultado))
print('El numero de clusters optimo es: '+str(min(resultado, key=resultado.get)))


141
El numero de clusters optimo es: 3


### Metodo del Codo

In [13]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [14]:
max_clusters=len(data)
clusters=range(2,max_clusters) #clusters a partir de 2

In [15]:
max_clust=len(data)
clusters=range(2,max_clust)#importante crear mas de 2 clusters para el metodo de silueta y codo
ssw=[] #suma de los cuadrados totales

for k in clusters:
    kmean_model=KMeans(n_clusters=k).fit(data)
    
    ssw_k=sum(np.min(cdist(data,kmean_model.cluster_centers_,'euclidean'),axis=1))
    ssw.append(ssw_k)
    print('Test con numero de clusters: '+str(k))    

Test con numero de clusters: 2
Test con numero de clusters: 3
Test con numero de clusters: 4
Test con numero de clusters: 5
Test con numero de clusters: 6
Test con numero de clusters: 7
Test con numero de clusters: 8
Test con numero de clusters: 9
Test con numero de clusters: 10
Test con numero de clusters: 11
Test con numero de clusters: 12
Test con numero de clusters: 13
Test con numero de clusters: 14
Test con numero de clusters: 15
Test con numero de clusters: 16
Test con numero de clusters: 17
Test con numero de clusters: 18
Test con numero de clusters: 19
Test con numero de clusters: 20
Test con numero de clusters: 21
Test con numero de clusters: 22
Test con numero de clusters: 23
Test con numero de clusters: 24
Test con numero de clusters: 25
Test con numero de clusters: 26
Test con numero de clusters: 27
Test con numero de clusters: 28
Test con numero de clusters: 29
Test con numero de clusters: 30
Test con numero de clusters: 31
Test con numero de clusters: 32
Test con numero 

KeyboardInterrupt: 

In [16]:
ssw2=np.diff(ssw) 
ssw2=[abs(num) for num in ssw2]

max=0

for index_i,i in enumerate(ssw2):
    if(i>max):
        max=i
        idx=index_i
    for index_j,j in enumerate(ssw2):
        if(j>max):
            max=j
            idx=index_j          
print('Numero optimo de clusters: '+str(clusters[idx+1]))

Numero optimo de clusters: 3
